In [1]:
from typing import Tuple, List, Dict
from datetime import datetime
import random
import pandas as pd

class User(object):
    
    def __init__(self, id_user: float, nick: str, name: str, bizum: float, location: str):
        self._id_user = id_user
        self._name = name
        self._nick = nick
        self._bizum = bizum
        self._location = location
        self._password = 0
        self._configured = 0
    
    def set_password(self, password: str) -> None:
        #self._password = str(input('Introduce your password of 5 digits: '))
        self._password = password
        if len(self._password) != 5:
            raise ValueError('Your password is not constructed using 5 digits, repeat again')
        self._configured = 1
        print('Password accepted')
        
    def _validate_password(self):
        if self._configured == 1:
            return self._password
        else: 
            print('You are not validated in our datebase')
    
    def user_configured(self):
        if self._configured == 1:
            return True
        else:
            return False
    
    def check_profile(self) -> Tuple[str, str, str, str]:
        return self._id_user, self._nick, self._name, self._location
    
    def obtain_bizum(self, nick:str, password: str) -> float:
        if self._nick == nick:
            if self._password == password:
                return self._bizum
            else:
                raise ValueError('Your password is incorrect')
        else:
            raise ValueError('This nick not exist')
               
class Ad(object):
    
    
    def __init__(self, id_ad: str, company: str, cost_per_visit : float):
        self._id_ad = id_ad
        self._company = company
        self._cost_per_visit = cost_per_visit
        self._users_seen = set()
        self._number_visits = len(self._users_seen)
    
    def visualize(self, user: User)-> None:                   #An user can only register one visualization
        #self._users_seen.append((user.check_id_name()[0])
        self._users_seen.add(user)  #Creo una lista de objetos Usuarios
        self._number_visits = len(self._users_seen)
        
    def show_properties(self):
        return [self._id_ad, self._company, self._cost_per_visit, self._number_visits]
    
    def export_users(self):
        return list(self._users_seen)
    
    
        
class Raffle(object):
    
    
    def __init__(self, id_raffle: str, name: str, ad : Ad, duration : float = 60):
        self._id_raffle = id_raffle
        self._name_raffle = name
        self._ad = ad   #Ojeto anuncio Asociado
        self._id_ad = ad.show_properties()[0]
        self._started = 0
        self._finished = 0
        
        self._time_duration = duration #Seconds of a day
        self._time_start = 0
        self._time_raffle = 0
        self._time_stop = 0
             
        
        self._prize_money = 0
        self._participants = list()
        self._id_participants = list()
        self._number_winners = 1
        self._id_winners = list()
        self._winners = list()
        
        
      
    def start_zerotime_raffle(self):
        self._time_start = datetime.now()
        self._time_raffle = self._time_duration + datetime.timestamp(self._time_start) #Add the duration
        self._time_stop = datetime.fromtimestamp(self._time_raffle)  
        self._started = 1
        
    def get_ad_associated(self):
        return self._ad
    
    def calculate_prize_money(self):
        if self.show_availability():
            self._prize_money = float(self._ad.show_properties()[2]) * float(self._ad.show_properties()[3])
            return self._prize_money
        else:
            print('The raffle has not started, the ad is not available')
    
        
    def show_properties(self):
        print(' The prize money is: {}'.format(self._prize_money),'\n',
         'The duration of the raffle is {} seconds'.format(self._time_duration), '\n',
         'It finishes at {}'.format(self._time_stop), '\n',
         'The number of winners is {}'.format(self._number_winners))
    
    def get_properties(self):
        properties_dict = {'name':  self._name_raffle,  'money': self._prize_money, 'id_raffle': self._id_raffle,
                           'duration': self._time_duration, 'time_raffle': self._time_stop,
                           'number_winners': self._number_winners}
        return properties_dict
        
    def show_availability(self):
        return self._started == 1
    
    def get_finished(self):
        return self._finished == 1
    
    def refresh_raffle(self):
        if (self._started == 1) & (datetime.now() > self._time_stop):
            self.raffle_select_winners()
        
    def raffle_select_winners(self):
        if self._finished == 1:    #I protect the method to avoid to repeat the specific raffle
            pass
            #return self._winners
        else:
            self._participants = self._ad.export_users()
            self._id_participants = [x.check_profile()[0] for x in self._participants]
            self._id_winners = random.sample(self._id_participants, k=self._number_winners)
            self._winners = [x for x in self._participants if x.check_profile()[0] in self._id_winners]
            self._finished = 1
        
            #return self._winners
    
    def announce_winners(self):  #Here will be the graphics of the raffle, this method have to be automatic when the app starts
        
        if self._finished == 1:
            print('The winners are {} from {} with a prize per winner of {} €'\
                  .format([x.check_profile()[2] for x in self._winners], 
                          [x.check_profile()[3] for x in self._winners],
                          self._prize_money / self._number_winners))
            
        elif self._started == 1:
            time_check = datetime.timestamp(datetime.now())
            remain_time = self._time_raffle - time_check
            print('The raffle has not been finished, the winners will be choosed in {} hours, {} minutes, {} seconds'\
                  .format(round(remain_time / 60 /60), round(remain_time / 60),
                          remain_time - round(round(remain_time / 60) * 60)))
        
        else:
            print('The raffle has not started')
        
    def get_winners(self):
        return self._winners
        
class App(object):  #An object of this class has to be used by any user, more than one users can be connected at the same time
                    #However each user is using the App in his phone, so is opening a new sentence of App
                    #Only activate one time for user as the installation
        
    def __init__(self,  list_users: [User], list_ads : [Ad], list_raffles: [Raffle]):
        
        self._users = {user.check_profile()[0] : user for user in list_users}
        self._users_nick_id = {user.check_profile()[1] : user.check_profile()[0] for user in list_users}
        self._ads = list_ads
        self._raffles = list_raffles
        self._main_user = object()
        self._available_raffles = list()
        self._finished_raffles = list()
        self._user_connected = 0
        self._raffle_prized_list= list()
        self._prize = 0
        self._winners_raffle_prize = list()
        self._winners = list()
        self._raffles_win = list()
        self._raffles_final_prize_euros = list()
        
        
    def connect(self, main_nick: str, password: str):     #Lo pasaría el usuario
        if self._user_connected == 1:
            print('You are already connected {}'.format(main_nick))
            self.refresh()
        else:
            #password = input('Introduce your password: ')
            User = self._users[self._users_nick_id[main_nick]]
            if password == User._validate_password():
                self._main_user = User
                self._user_connected = 1
                print('You are connected now')
                self.refresh()
            
    def disconnect(self):
        if self._user_connected == 1:
            self._user_connected = 0
        else:
            print('You are not connected already')
        
        
    def see_available_raffles(self): #Generar grafica datos 
        self.refresh()
        
        if self._user_connected == 1:                   
            screen_raffle = [x.get_properties() for x in self._available_raffles.values() \
                             if x not in self._finished_raffles.values()] #Con este if evitamos que vean sorteos ya premiados
            return screen_raffle
        else:
            print('You are not logged in the App')
            
    def choose_raffle(self, name_raffle: str):   #You select with a push button the raffle
        self.refresh()
        
        if self._user_connected == 1:
            raffle_choosen = self._available_raffles[name_raffle] #If I want to choose with the Id I need to change how I 
                                                                    #create the dictionary    
            #question = input('Do you want to participate in the raffle? Write (y/n) ')
            question = 'y'
            
            if question == 'y':
                self._participate_raffle(raffle_choosen)
                print('You are participating in the {}.'.format(name_raffle))
                return raffle_choosen.get_properties()
            else:
                print('Okey, is your decision')
        
        else:
            print('You are not logged in the App')
    
    
    def _participate_raffle(self, raffle_choosen: Raffle):  #I want this method to be private and we can only access since 
        if self._user_connected == 1:                        # choose_raffle
            ad_associated = raffle_choosen.get_ad_associated()
            ad_associated.visualize(user = self._main_user)
        
        else:
            print('You are not logged in the App')
            
    def refresh(self):
        
        [raffle.refresh_raffle() for raffle in self._raffles if raffle.show_availability()]
        
        self._available_raffles = {raffle.get_properties()['name']: raffle for raffle in self._raffles\
                                        if raffle.show_availability()}
                
        self._finished_raffles = {raffle.get_properties()['name']: raffle for raffle in self._raffles\
                                        if (raffle.show_availability() & raffle.get_finished())}
            
        self._winners_raffle_prize= [(winners, raffle, raffle.calculate_prize_money()) for raffle \
                                        in self._finished_raffles.values() for winners in raffle.get_winners()] 
                                        #2 comprensiones de lista
        
        #Estos 3 vectores estan ordenados a la vez
        self._winners = [tuples[0].check_profile()[2] for tuples in self._winners_raffle_prize]   #Select name
        self._raffles_win = [tuples[1].get_properties()['name'] for tuples in self._winners_raffle_prize]
        self._raffles_final_prize_euros = [tuples[2] for tuples in self._winners_raffle_prize]
        #Compongo un dataframe con estos 3 vectores
        self._rank_winner = pd.DataFrame({'Winners':self._winners, 'Raffle': self._raffles_win, 
                                         'Prize (€)':self._raffles_final_prize_euros})
                                        
    
    def collect_prize(self, password: str):   
        self.refresh()
        if self._user_connected == 1:
            if self._main_user.check_profile()[2] in self._winners:
                
                raffle_prize = [raffle for raffle in self._finished_raffles.values() \
                                if (raffle.get_winners() == self._main_user)]
                
                
                for raffle in raffle_prize:
                    if raffle in self._raffle_prized_list:
                        self._prize = 0
                    else:
                        self._prize += raffle.calculate_prize_money()
                        self._raffle_prized_list = raffle_prized_list.append(raffle)
                
                if self._prize == 0:
                    print('Your prize has been paid and you have not win any more prizes')
                else:
                    #password = str(input('Introduce your password'))
                    self._pay_bizum(self._main_user.check_profile()[1], password, self._prize)
            else:
                print('You have not win any prize yet')
        else:
            print('You are not logged in the App')
    
    def see_main_user(self):
        return self._main_user.check_profile()
            
    def _pay_bizum(self, nick:str, password: str, money: float):
        bizum = self._main_user.obtain_bizum(nick, password)
        print('The prize of {} € has been paid to the user {} of bizum {}'.format(money, nick, bizum))
        
    def see_winners(self):
        self.refresh()
        return  self._rank_winner